In [1]:
#jupyter labextension install jupyterlab-plotly
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
%matplotlib inline
from ExKMC.Tree import Tree
from IPython.display import Image

np.random.seed(1)
import random
import sys

import sklearn
import sklearn.ensemble
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import minmax_scale

import shap
import joblib as jbl

import lime
import lime.lime_tabular

from anchor import utils
from anchor import anchor_tabular

import json
import warnings

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'
import kaleido
import os
import img2pdf

/tmp/ipykernel_116446/3954359919.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [10]:
## Generating 150k Dataset with random instances from yes and no datasets
# path_datasets = "../datasets/"
# data_yes = pd.read_csv(path_datasets+"Dataset_99k_yes.csv")
# data_no = pd.read_csv(path_datasets+"Dataset_75k_no.csv")

# data_yes = data_yes.set_index("id_")
# data_no = data_no.rename(columns={"id": "id_"})
# data_no = data_no.set_index("id_")

# sample_data_yes = data_yes.sample(n=75000)
# sample_data_no = data_no.sample(n=75000)

# training_samples = pd.concat([sample_data_yes, sample_data_no], axis=0)
# display(training_samples)
# training_samples.to_csv(path_datasets+"Random_Generated_Dataset_150k.csv")

,methodAnonymousClassesQty,methodAssignmentsQty,methodCbo,methodComparisonsQty,methodLambdasQty,methodLoc,methodLoopQty,methodMathOperationsQty,methodMaxNestedBlocks,methodNumbersQty,...,methodRfc,methodStringLiteralsQty,methodSubClassesQty,methodTryCatchQty,methodUniqueWordsQty,methodVariablesQty,methodWmc,bugFixCount,refactoringsInvolved,y
id_,,,,,,,,,,,,,,,,,,,,,
27308,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,4,0,1,0,0,1
19994,0,10,1,0,0,25,2,2,3,0,...,8,7,0,0,32,8,6,3,5,1
29413,0,2,5,0,0,15,0,0,2,0,...,8,1,0,1,16,1,3,1,1,1
52702,0,23,11,2,0,57,2,1,3,4,...,34,1,0,0,59,14,13,4,10,1
41416,0,0,1,0,0,4,0,0,0,0,...,1,0,0,0,6,0,2,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0,0,4,0,0,3,0,0,0,0,...,0,1,0,0,13,0,1,0,7,0
1057847,1,0,3,0,0,13,0,0,1,0,...,0,0,0,0,10,0,1,0,0,0
973440,0,1,3,0,0,3,1,0,0,1,...,5,0,0,0,10,1,3,26,9,0


In [7]:
# importing training dataset
path_datasets = "../datasets/"
training_samples = pd.read_csv(path_datasets+"Random_Generated_Dataset_150k.csv")
data = training_samples.set_index("id_")
display(data)

# splitting train and test
X = data.drop(['y'], axis=1)
y = data['y']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

# training random forest model
forest = RandomForestClassifier(random_state=0)
forest.fit(X, y)

# getting list of feature names
feature_names = list(X.columns)

,methodAnonymousClassesQty,methodAssignmentsQty,methodCbo,methodComparisonsQty,methodLambdasQty,methodLoc,methodLoopQty,methodMathOperationsQty,methodMaxNestedBlocks,methodNumbersQty,...,methodRfc,methodStringLiteralsQty,methodSubClassesQty,methodTryCatchQty,methodUniqueWordsQty,methodVariablesQty,methodWmc,bugFixCount,refactoringsInvolved,y
id_,,,,,,,,,,,,,,,,,,,,,
27308,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,4,0,1,0,0,1
19994,0,10,1,0,0,25,2,2,3,0,...,8,7,0,0,32,8,6,3,5,1
29413,0,2,5,0,0,15,0,0,2,0,...,8,1,0,1,16,1,3,1,1,1
52702,0,23,11,2,0,57,2,1,3,4,...,34,1,0,0,59,14,13,4,10,1
41416,0,0,1,0,0,4,0,0,0,0,...,1,0,0,0,6,0,2,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0,0,4,0,0,3,0,0,0,0,...,0,1,0,0,13,0,1,0,7,0
1057847,1,0,3,0,0,13,0,0,1,0,...,0,0,0,0,10,0,1,0,0,0
973440,0,1,3,0,0,3,1,0,0,1,...,5,0,0,0,10,1,3,26,9,0


In [15]:
# Setting up SHAP
try:
    with open("./" + 'shap_explainer', 'rb') as f:
        explainer_shap = jbl.load(f)
except:
    explainer_shap = shap.TreeExplainer(forest)
    with open("./" + 'shap_explainer', 'wb') as f:
        jbl.dump(explainer_shap, f)

# Setting up LIME
explainer_lime = lime.lime_tabular.LimeTabularExplainer(X.values, feature_names=X.columns.values.tolist(), class_names=[0, 1], verbose=True, mode='classification',  discretize_continuous=True)

# Setting up Anchors
explainer_anchors = anchor_tabular.AnchorTabularExplainer(
    [0, 1],
    X.columns.values.tolist(),
    X.values,
    {})

In [61]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# function to help sorting features
def compare_shap_feature_weights(feature_value):
    # highest weights first
    return -feature_value['feature_weight']

def scale_weights(features_weights, features_exp):
    # scale weights to sum to 1
    np_features_weights = np.asarray(features_weights)
    scaled_features_weights = np_features_weights / np_features_weights.sum()
    # add values to dict
    for i in range(0, len(features_exp)):
        features_exp[i]['feature_weight'] = scaled_features_weights[i]
    return features_exp

# export shap features' explanations
def export_shap_exp(row, feature_names, shap_values, refactor_bool):
    # get shap values to refactor instance
    shap_values_to_refactor = shap_values[1]
    shap_output = dict()
    
    features_exp = []
    features_weights = []
    for i in range(0, len(feature_names)):
        condition = shap_values_to_refactor[i] > 0 if refactor_bool else shap_values_to_refactor[i] < 0
        # shap_values has positive values (to refactor) and negative ones (not to refactor)
        if condition:
            f = dict()
            feature_name = feature_names[i]
            f['feature_name'] = feature_name
            f['feature_value'] = int(row[feature_name])
            features_weights.append(shap_values_to_refactor[i])
            f['feature_ranges'] = None
            features_exp.append(f)
    # scale weights to sum to 1
    scaled_features_exp = scale_weights(features_weights, features_exp)
    # sort features by feature weight
    sorted_features_exp = sorted(scaled_features_exp, key=compare_shap_feature_weights)
    # append feature ranking after sort
    rank = 1
    for f in sorted_features_exp:
        f['feature_rank'] = rank
        rank += 1
    shap_output['features'] = sorted_features_exp
    return shap_output

# export lime features' explanations
def export_lime_exp(row, feature_names, exp_lime, refactor_bool):
    lime_output = dict()
    # general instance indices
    lime_output['intercept'] = exp_lime.intercept[1]
    lime_output['local_prediction'] = exp_lime.local_pred[0]
    lime_features = exp_lime.as_list()
    # features' values
    features_exp = []
    features_weights = []
    rank = 1
    for value in lime_features:
        condition = value[1] > 0 if refactor_bool else value[1] < 0
        # value[1] (feature_weight) has positive values (to refactor) and negative ones (not to refactor)
        if condition:
            f = dict()
            # extract feature name from feature ranges string
            any((feature_name := substring) in value[0] for substring in feature_names)
            f['feature_name'] = feature_name
            f['feature_value'] = int(row[feature_name])
            features_weights.append(value[1])
            f['feature_ranges'] = value[0]
            f['feature_rank'] = rank # feature's order of priority in explainer's result
            features_exp.append(f)
            rank += 1
    # scale weights to sum to 1
    scaled_features_exp = scale_weights(features_weights, features_exp)
    lime_output['features'] = scaled_features_exp
    return lime_output

# export anchors features' explanations
def export_anchors_exp(row, feature_names, anchors_exp, refactor_bool):
    anchors_output = dict()
    # general instance indices
    anchors_output['precision'] = anchors_exp.precision()
    anchors_output['coverage'] = anchors_exp.coverage()
    # features' values
    features_exp = []
    features_weights = []
    rank = 1
    for i in range(0, len(anchors_exp.names())):
        f = dict()
        # extract feature name from anchors' names string
        any((feature_name := substring) in str(anchors_exp.names()[i]) for substring in feature_names)
        f['feature_name'] = feature_name
        f['feature_value'] = int(row[feature_name])
        weight = (anchors_exp.precision(i)*anchors_exp.coverage(i))/anchors_exp.precision()
        features_weights.append(weight)
        f['feature_weight'] = None
        f['feature_ranges'] = anchors_exp.names()[i]
        f['feature_rank'] = rank # feature's order of priority in explainer's result
        features_exp.append(f)
        rank += 1
    # scale weights to sum to 1
    scaled_features_exp = scale_weights(features_weights, features_exp)
    anchors_output['features'] = features_exp
    return anchors_output

def run_and_export_explainers(samples, i, refactor_bool):
    feature_names = list(samples.columns)
    
    #SHAP
    row = samples.iloc[i]
    shap_values = explainer_shap.shap_values(row)
    shap_output = export_shap_exp(row, feature_names, shap_values, refactor_bool)

    #LIME
    exp_lime = explainer_lime.explain_instance(samples.values[i], forest.predict_proba, num_features=8)
    lime_output = export_lime_exp(row, feature_names, exp_lime, refactor_bool)

    #ANCHORS
    exp_anchors = explainer_anchors.explain_instance(samples.values[i], forest.predict, threshold=0.95)
    anchors_output = export_anchors_exp(row, feature_names, exp_anchors, refactor_bool)
    
    return [shap_output, lime_output, anchors_output]

def export_other_explanations(samples, indexes, json_name):
    explanations = dict()
    for i in range(0, len(samples)):
        json_output = dict()
        # gives values of random forest predictions
        predict = forest.predict_proba(samples.iloc[[i]])[0]
        json_output['forest_prediction'] = {'not to refactor': predict[0], 'refactor': predict[1]}
        # var to indicate if instance must be refactored or not
        refactor_bool = 1 if predict[1] >= 0.5 else 0
        # calls funtions that run and export shap, lime and anchors explanations
        json_output['shap'], json_output['lime'], json_output['anchors'] = run_and_export_explainers(samples, i, refactor_bool)
        # puts explanations in each instance index
        explanations[int(indexes[i])] = json_output
    #export explanations to json file
    path = "other_approach_jsons"
    if not os.path.exists(path):
        os.makedirs(path)
    file_name = f"{path}/{json_name}_other_top_features_ranking.json"
    with open(file_name, "w") as outfile:
        json.dump(explanations, outfile)
    return explanations

In [62]:
def extract_feature_names(json_features):
    extracted_feature_names = []
    for f in json_features:
        extracted_feature_names.append(f['feature_name'])
    return extracted_feature_names

def get_info_by_feature_name(exp_f, feature_name):
    feature_obj = next(x for x in exp_f if x["feature_name"] == feature_name)
    return feature_obj['feature_value'], feature_obj['feature_weight'], feature_obj['feature_ranges'], feature_obj['feature_rank']

def compare_feature_weights(feature_value, order):    
    if order[0]['explainer'] in feature_value and order[1]['explainer'] in feature_value and order[2]['explainer'] in feature_value:
        return feature_value[order[0]['explainer']], feature_value[order[1]['explainer']], feature_value[order[2]['explainer']]
    elif order[0]['explainer'] in feature_value and order[1]['explainer'] in feature_value:
        return feature_value[order[0]['explainer']], feature_value[order[1]['explainer']]
    elif order[0]['explainer'] in feature_value and order[2]['explainer'] in feature_value:
        return feature_value[order[0]['explainer']], feature_value[order[2]['explainer']]
    elif order[1]['explainer'] in feature_value and order[2]['explainer'] in feature_value:
        return feature_value[order[1]['explainer']], feature_value[order[2]['explainer']]

def combine_feature_explanations(feature_names, instance, priority_order):
    sum_priority_order = 0
    for p in priority_order:
        sum_priority_order = sum_priority_order + priority_order[p]['priority_weight']
        
    shap_f = instance['shap']['features']
    lime_f = instance['lime']['features']
    anchors_f = instance['anchors']['features']

    shap_f_names = extract_feature_names(shap_f)
    lime_f_names = extract_feature_names(lime_f)
    anchors_f_names = extract_feature_names(anchors_f)

    combine_all, combine_shap_lime, combine_shap_anchors, combine_lime_anchors = ([] for i in range(4))
    compiled_combinations = dict()
    for f in feature_names:
        # for each configuration of combination, gets feature name and explainers ranking
        # when finds feature in shap, lime and anchors
        if f in shap_f_names and f in lime_f_names and f in anchors_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, anchors_weight, anchors_ranges, anchors_rank = get_info_by_feature_name(anchors_f, f)
            _, shap_weight, _, shap_rank = get_info_by_feature_name(shap_f, f)
            _, lime_weight, lime_ranges, lime_rank = get_info_by_feature_name(lime_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_anchors'] = anchors_rank
            feature_summary['rank_shap'] = shap_rank
            feature_summary['rank_lime'] = lime_rank
            feature_summary['weight_anchors'] = anchors_weight
            feature_summary['weight_shap'] = shap_weight
            feature_summary['weight_lime'] = lime_weight
            feature_summary['avg_weight'] = (2*shap_weight+lime_weight+3*anchors_weight)/6
            feature_summary['range_anchors'] = anchors_ranges
            feature_summary['range_lime'] = lime_ranges
            feature_summary['agreement_index'] = (priority_order[0]['priority_weight']+priority_order[1]['priority_weight']+priority_order[2]['priority_weight'])/sum_priority_order
            combine_all.append(feature_summary)
        # when finds feature in shap and lime
        elif f in shap_f_names and f in lime_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, shap_weight, _, shap_rank = get_info_by_feature_name(shap_f, f)
            _, lime_weight, lime_ranges, lime_rank = get_info_by_feature_name(lime_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_shap'] = shap_rank
            feature_summary['rank_lime'] = lime_rank
            feature_summary['weight_shap'] = shap_weight
            feature_summary['weight_lime'] = lime_weight
            feature_summary['avg_weight'] = (2*shap_weight+lime_weight)/3
            feature_summary['range_lime'] = lime_ranges
            feature_summary['agreement_index'] = (priority_order[1]['priority_weight']+priority_order[2]['priority_weight'])/sum_priority_order
            combine_shap_lime.append(feature_summary)
        # when finds feature in shap and anchors
        elif f in shap_f_names and f in anchors_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, anchors_weight, anchors_ranges, anchors_rank = get_info_by_feature_name(anchors_f, f)
            _, shap_weight, _, shap_rank = get_info_by_feature_name(shap_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_anchors'] = anchors_rank
            feature_summary['rank_shap'] = shap_rank
            feature_summary['weight_anchors'] = anchors_weight
            feature_summary['weight_shap'] = shap_weight
            feature_summary['avg_weight'] = (2*shap_weight+3*anchors_weight)/5
            feature_summary['range_anchors'] = anchors_ranges
            feature_summary['agreement_index'] = (priority_order[0]['priority_weight']+priority_order[1]['priority_weight'])/sum_priority_order
            combine_shap_anchors.append(feature_summary)
        # when finds feature in lime and anchors
        elif f in lime_f_names and f in anchors_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, anchors_weight, anchors_ranges, anchors_rank = get_info_by_feature_name(anchors_f, f)
            _, lime_weight, lime_ranges, lime_rank = get_info_by_feature_name(lime_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_anchors'] = anchors_rank
            feature_summary['rank_lime'] = lime_rank
            feature_summary['weight_anchors'] = anchors_weight
            feature_summary['weight_lime'] = lime_weight
            feature_summary['avg_weight'] = (lime_weight+3*anchors_weight)/4
            feature_summary['range_anchors'] = anchors_ranges
            feature_summary['range_lime'] = lime_ranges
            feature_summary['agreement_index'] = (priority_order[0]['priority_weight']+priority_order[2]['priority_weight'])/sum_priority_order
            combine_lime_anchors.append(feature_summary)

    # order each list by importance (1st anchors, 2nd shap, 3rd lime)
    compiled_combinations['combine_all'] = sorted(combine_all, key=lambda feature_value: compare_feature_weights(feature_value, priority_order))
    compiled_combinations['combine_shap_lime'] = sorted(combine_shap_lime, key=lambda feature_value: compare_feature_weights(feature_value, priority_order))
    compiled_combinations['combine_shap_anchors'] = sorted(combine_shap_anchors, key=lambda feature_value: compare_feature_weights(feature_value, priority_order))
    compiled_combinations['combine_lime_anchors'] = sorted(combine_lime_anchors, key=lambda feature_value: compare_feature_weights(feature_value, priority_order))
    
    return compiled_combinations

# 1st: get combinations between all the 3 explainers
# 2nd: get combinations between 2 explainers
#      decision between explainers will have
#      priority -> 1 - anchors, 2 - shap, 3 - lime
def combine_top_k_features(feature_names, json_out, k, priority_order, json_name):
    final_explanations = dict()
    # run each instance
    for i in json_out:
        combinations = combine_feature_explanations(feature_names, json_out[i], priority_order)
        # 1st step of algorithm
        # get combinations between all explainers
        final_k_features = combinations['combine_all'][0:k]
        # verify how many features misses to complete the x top features
        missing_features = k - len(final_k_features)
        
        if missing_features:
            # get x reamining features from each of combinations between 2 explainers
            i_shap_lime_f = combinations['combine_shap_lime'][0:missing_features]
            i_shap_anchors_f = combinations['combine_shap_anchors'][0:missing_features]
            i_lime_anchors_f = combinations['combine_lime_anchors'][0:missing_features]
            # concat lists
            remaining_features = i_shap_lime_f+i_shap_anchors_f+i_lime_anchors_f
            # 2st step of algorithm
            # sort concatenated list
            sorted_remaining_features = sorted(remaining_features, key=lambda feature_value: compare_feature_weights(feature_value, priority_order))
            # concat features selected
            final_k_features = final_k_features + sorted_remaining_features[0:missing_features]

        instance_final_exp = dict()
        instance_final_exp['forest_prediction_to_refactor'] = json_out[i]['forest_prediction']['refactor']
        top_n_features_string = 'top_'+str(k)+'_features'
        instance_final_exp[top_n_features_string] = final_k_features
        
        final_explanations[i] = instance_final_exp

    #export explanations to json file
    path = "our_approach_jsons"
    if not os.path.exists(path):
        os.makedirs(path)
    file_name = f"{path}/our_top_{str(k)}_ranking_{json_name}.json"
    with open(file_name, "w") as outfile:
        json.dump(final_explanations, outfile)
        
    return final_explanations

In [81]:
def get_round_percentage(weight):
    return round(weight*100, 1)
    
def get_top_k_features_from_approach(k, approach_exp, write_weight, write_agreement_index):
    top_k_approach = [None] * k
    k_approach = k if len(approach_exp) >= k else len(approach_exp)
    for feature in range(0, k_approach):
        exp_f = approach_exp[feature]
        feature_name = exp_f["feature_name"]
        avg_weight = f" ({get_round_percentage(exp_f[write_weight])}%" if write_weight else ""
        agreement_index = f" - {get_round_percentage(exp_f[write_agreement_index])}%)" if write_agreement_index else ")"
        top_k_approach[feature] = feature_name+avg_weight+agreement_index
    return top_k_approach

def generate_top_k_ranking_for_each_approach(k, our_approach_explanations, other_explanations, json_name):
    general_top_k = dict()
    
    for i in our_approach_explanations:
        top_k_each_instance = dict()

        top_k_each_instance["forest_pred"] = our_approach_explanations[i]["forest_prediction_to_refactor"]
        top_k_each_instance["forest_pred"] = our_approach_explanations[i]["forest_prediction_to_refactor"]
        top_k_each_instance["our_approach"] = get_top_k_features_from_approach(k, our_approach_explanations[i]["top_5_features"], "avg_weight", "agreement_index")
        top_k_each_instance["shap_approach"] = get_top_k_features_from_approach(k, other_explanations[i]["shap"]["features"], "feature_weight", 0)
        top_k_each_instance["lime_approach"] = get_top_k_features_from_approach(k, other_explanations[i]["lime"]["features"], "feature_weight", 0)
        top_k_each_instance["anchors_approach"] = get_top_k_features_from_approach(k, other_explanations[i]["anchors"]["features"], "feature_weight", 0)
    
        general_top_k[str(i)] = top_k_each_instance
    #export explanations to json file
    path = "top_k_rankings_jsons"
    if not os.path.exists(path):
        os.makedirs(path)
    file_name = f"{path}/general_top_{str(k)}_ranking_{json_name}.json"
    with open(file_name, "w") as outfile:
        json.dump(general_top_k, outfile)
    return general_top_k

In [67]:
def plot_explanation_rankings_table(instance, df):
    positions = ['<b>1</b>', '<b>2</b>', '<b>3</b>','<b>4</b>', '<b>5</b>']
    rank_colors = ['#FFB0B0', '#FFC89F', '#FFF3A0', '#BEFF91', '#A8DDFF']
    instance_info = f"<b>Instance {instance} - {df.forest_pred[0]}</b>"
    
    fig = go.Figure(data=[go.Table(
      columnwidth = [200, 150,300],
      header=dict(
        values=[instance_info, "<b>Rank Position</b>", "SHAP (weight)", "LIME (weight)", "Anchors(weight)", "Our Approach (weight - agreement index)"],
        line_color='darkslategray', fill_color='white',
        align='center', font=dict(color='black', size=14)
      ),
      cells=dict(
        values=[[], positions, df.shap_approach, df.lime_approach, df.anchors_approach, df.our_approach],
        line_color='darkslategray', fill=dict(color=['white', rank_colors]),
        align='center', font=dict(color='black', size=12)
      ))
    ])
    # fig.show(renderer="png", width=1150, height=500)
    fig_formatted = fig.to_image(format="png", width=1150, height=500, engine="kaleido")
    return fig_formatted

def export_plot_explanation_rankings(top_k_rankings, pdf_name):
    # if not os.path.exists("plot_images"):
    #     os.mkdir("plot_images")
    rankings_plots_img_bytes = []
    for i in top_k_rankings:
        rankings = pd.DataFrame(top_k_rankings[i])
        plot_img_bytes = plot_explanation_rankings_table(i, rankings)
        rankings_plots_img_bytes.append(plot_img_bytes)
    
    rankings_plots_pdf = img2pdf.convert(rankings_plots_img_bytes)
    # save to pdf
    path = "top_k_rankings_pdfs"
    if not os.path.exists(path):
        os.makedirs(path)
    pdf_name = f"{path}/{pdf_name}"
    with open(pdf_name, "wb") as file:
        file.write(rankings_plots_pdf)

In [4]:
def get_formatted_data_and_indexes(dataset_name, id_column, target_column = 0):
    # importing dataset
    dataset = f"datasets/{dataset_name}"
    data = pd.read_csv(dataset)
    # getting dataset ids
    data_indexes = data[id_column].to_list()
    data = data.drop([target_column], axis=1) if target_column else data
    data = data.set_index(id_column)
    return data, data_indexes

def export_top_k_ranking(k, priority_order, samples, samples_indexes, samples_name):
    feature_names = list(samples.columns)
    
    other_explanations = export_other_explanations(samples, samples_indexes, samples_name)
    
    our_explanations = combine_top_k_features(feature_names, other_explanations, k, priority_order, samples_name)
        
    all_top_k_rankings = generate_top_k_ranking_for_each_approach(k, our_explanations, other_explanations, samples_name)
    pdf_name = f"{samples_name}_top_k_rankings.pdf"
    export_plot_explanation_rankings(all_top_k_rankings, pdf_name)

In [10]:
k = 5

priority_order = {0: {'explainer': 'rank_anchors', 'priority_weight': 3},
         1: {'explainer': 'rank_shap', 'priority_weight': 2},
         2: {'explainer': 'rank_lime', 'priority_weight': 1}}

In [2]:
def separate_samples_by_category(cluster, samples_indexes):
    negative_predict, predict_50_60, predict_60_90, predict_90 = ([] for i in range(4))
    for i in range(0, len(cluster)):
        predict = forest.predict_proba(cluster.iloc[[i]])[0][1]
        if predict < 0.5:
            negative_predict.append(samples_indexes[i])
        elif predict >= 0.5 and predict < 0.6:
            predict_50_60.append(samples_indexes[i])
        elif predict >= 0.6 and predict < 0.9:
            predict_60_90.append(samples_indexes[i])
        elif predict >= 0.9:
            predict_90.append(samples_indexes[i])
    cluster = cluster.drop(negative_predict, axis=0)
    # get 5 random samples from each group if there are more than 4 instances in the group
    samples_predict_50_60 = random.sample(predict_50_60, 5) if len(predict_50_60) > 4 else predict_50_60
    samples_predict_60_90 = random.sample(predict_60_90, 5) if len(predict_60_90) > 4 else predict_60_90
    samples_predict_90 = random.sample(predict_90, 5) if len(predict_90) > 4 else predict_90
    print("Negative predictions: ", len(negative_predict), "\nPredictions beetwen 0.5 and 0.6: ", len(predict_50_60), "\nPredictions beetwen 0.6 and 0.9: ", len(predict_60_90), "\nPredictions higher than 0.9: ", len(predict_90))
    return [samples_predict_50_60, samples_predict_60_90, samples_predict_90]

def concat_groups_of_samples(cluster, cluster_indexes):
    samples_predict_50_60 = cluster.loc[cluster_indexes[0]]
    samples_predict_60_90 = cluster.loc[cluster_indexes[1]]
    samples_predict_90 = cluster.loc[cluster_indexes[2]]

    samples_cluster = pd.concat([samples_predict_50_60, samples_predict_60_90, samples_predict_90])
    simplified_cluster_indexes = np.concatenate(cluster_indexes)
    return samples_cluster, simplified_cluster_indexes

In [33]:
# importing cluster0 to explain
cluster0, samples_indexes_cluster0 = get_formatted_data_and_indexes("2508_Cluster_00.csv", "id_", "cluster")

# importing cluster1 to explain
cluster1, samples_indexes_cluster1 = get_formatted_data_and_indexes("2508_Cluster_01.csv", "id_", "cluster")

# separate 5 samples for each category:
# G1 - RF prediction between 50 - 60%
# G2 - RF prediction between 60 - 90%
# G3 - RF prediction higher than 90%
print("Cluster 0")

predict_90_c0 = []
predict_90_c1 = []
for i in range(0, len(cluster0)):
    predict = forest.predict_proba(cluster0.iloc[[i]])[0][1]
    if predict >= 0.9:
        predict_90_c0.append(samples_indexes_cluster0[i])

for i in range(0, len(cluster1)):
    predict = forest.predict_proba(cluster1.iloc[[i]])[0][1]
    if predict >= 0.9:
        predict_90_c1.append(samples_indexes_cluster1[i])
# get 5 random samples from each group if there are more than 4 instances in the group
samples_predict_90_c0 = random.sample(predict_90_c0, 1000) if len(predict_90_c0) > 1000 else predict_90_c0
samples_predict_90_c1 = random.sample(predict_90_c1, 1000) if len(predict_90_c1) > 1000 else predict_90_c1

Cluster 0


In [32]:
display(predict_90_c1)

[]

In [35]:
samples_1k_c0 =  cluster0.filter(items=samples_predict_90_c0, axis=0)

samples_1k_c1 =  cluster1.filter(items=samples_predict_90_c1, axis=0)

samples_1k_c0.to_csv(path_datasets+"1k_samples_cluster0.csv")
samples_1k_c1.to_csv(path_datasets+"1k_samples_cluster1.csv")

In [74]:
# importing cluster0 to explain
cluster0, samples_indexes_cluster0 = get_formatted_data_and_indexes("Cluster_00.csv", "id_", "cluster")

# importing cluster2 to explain
cluster2, samples_indexes_cluster2 = get_formatted_data_and_indexes("Cluster_02.csv", "id_", "cluster")

# separate 5 samples for each category:
# G1 - RF prediction between 50 - 60%
# G2 - RF prediction between 60 - 90%
# G3 - RF prediction higher than 90%
print("Cluster 0")
samples_indexes_cluster0 = separate_samples_by_category(cluster0, samples_indexes_cluster0)
print("Cluster 2")
samples_indexes_cluster2 = separate_samples_by_category(cluster2, samples_indexes_cluster2)

# concat samples in 1 dataframe to group the result in one pdf
samples_cluster0, simplified_indexes_cluster0 = concat_groups_of_samples(cluster0, samples_indexes_cluster0)
samples_cluster2, simplified_indexes_cluster2 = concat_groups_of_samples(cluster2, samples_indexes_cluster2)

export_top_k_ranking(k, priority_order, samples_cluster0, simplified_indexes_cluster0, "cluster0")
export_top_k_ranking(k, priority_order, samples_cluster2, simplified_indexes_cluster2, "cluster2")

Cluster 0
Negative predictions:  7 
Predictions beetwen 0.5 and 0.6:  6 
Predictions beetwen 0.6 and 0.9:  19 
Predictions higher than 0.9:  8
Cluster 2
Negative predictions:  0 
Predictions beetwen 0.5 and 0.6:  1 
Predictions beetwen 0.6 and 0.9:  17 
Predictions higher than 0.9:  22
Intercept 0.4873605783948534
Prediction_local [0.66259933]
Right: 0.55
Intercept 0.5011840383945643
Prediction_local [0.53962802]
Right: 0.55
Intercept 0.5805355663314902
Prediction_local [0.50627134]
Right: 0.5
Intercept 0.580889011377583
Prediction_local [0.5631048]
Right: 0.57
Intercept 0.5139634819292395
Prediction_local [0.46446561]
Right: 0.5467738095238095
Intercept 0.3955775689243477
Prediction_local [0.6840591]
Right: 0.85
Intercept 0.34645695009832767
Prediction_local [0.64117838]
Right: 0.82
Intercept 0.4871383177114124
Prediction_local [0.65026648]
Right: 0.84
Intercept 0.4484494604250755
Prediction_local [0.6333454]
Right: 0.8
Intercept 0.49527831497094865
Prediction_local [0.61050714]
Right

Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an a

Intercept 0.5111475532963863
Prediction_local [0.43521769]
Right: 0.54
Intercept 0.4801548129215474
Prediction_local [0.68868692]
Right: 0.79
Intercept 0.5908405492018381
Prediction_local [0.35192735]
Right: 0.89
Intercept 0.4996928032376775
Prediction_local [0.40872862]
Right: 0.66
Intercept 0.5102312900310897
Prediction_local [0.44313599]
Right: 0.85
Intercept 0.6424519892419184
Prediction_local [0.39099917]
Right: 0.67
Intercept 0.5182696017299134
Prediction_local [0.61402753]
Right: 0.95
Intercept 0.3887838645608035
Prediction_local [0.65537357]
Right: 0.94
Intercept 0.5141669132478637
Prediction_local [0.60722474]
Right: 0.97
Intercept 0.44089715621806497
Prediction_local [0.64206169]
Right: 0.94
Intercept 0.44847884602807886
Prediction_local [0.61163908]
Right: 0.99


Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an a

In [75]:
samples_10_positive, samples_indexes_10_positive = get_formatted_data_and_indexes("Amostra-Dataset.csv", "id_")
export_top_k_ranking(k, priority_order, samples_10_positive, samples_indexes_10_positive, "10_positive_samples")

Intercept 0.4283444302279355
Prediction_local [0.71331141]
Right: 0.9
Intercept 0.46961365883432077
Prediction_local [0.71598881]
Right: 0.91
Intercept 0.49597561251144723
Prediction_local [0.69204529]
Right: 0.96
Intercept 0.5044314747303343
Prediction_local [0.42422714]
Right: 0.83
Intercept 0.48655327051639125
Prediction_local [0.56694038]
Right: 0.95
Intercept 0.60623600150518
Prediction_local [0.4124607]
Right: 0.85
Intercept 0.5446801214617696
Prediction_local [0.67464846]
Right: 0.97
Intercept 0.4301668382291035
Prediction_local [0.61977351]
Right: 0.78
Intercept 0.4782421999773504
Prediction_local [0.77996678]
Right: 0.99
Intercept 0.5632574073576998
Prediction_local [0.39995814]
Right: 0.51


Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an a

In [76]:
samples_10_negative, samples_indexes_10_negative = get_formatted_data_and_indexes("NO_instances.csv", "id")
export_top_k_ranking(k, priority_order, samples_10_negative, samples_indexes_10_negative, "10_negative_samples")

Intercept 0.5478211902906257
Prediction_local [0.3536736]
Right: 0.15
Intercept 0.5419000555025999
Prediction_local [0.59292239]
Right: 0.31
Intercept 0.5775078880574468
Prediction_local [0.61488386]
Right: 0.35
Intercept 0.394180076931551
Prediction_local [0.58218739]
Right: 0.24
Intercept 0.5819128607954254
Prediction_local [0.49768791]
Right: 0.11
Intercept 0.5803049216510627
Prediction_local [0.34458158]
Right: 0.21
Intercept 0.5106273523682839
Prediction_local [0.57039349]
Right: 0.3
Intercept 0.45532283855038824
Prediction_local [0.69966663]
Right: 0.26
Intercept 0.4911524017628196
Prediction_local [0.73836011]
Right: 0.34
Intercept 0.513922246706817
Prediction_local [0.65912291]
Right: 0.18


Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an a

In [77]:
samples_yes_experiment, indexes_yes_experiment = get_formatted_data_and_indexes("Yes_Instances_Experiment.csv", "id", "y")
export_top_k_ranking(k, priority_order, samples_yes_experiment, indexes_yes_experiment, "yes_experiment")

samples_no_experiment, indexes_no_experiment = get_formatted_data_and_indexes("No_Instances_Experiment.csv", "id", "Y")
export_top_k_ranking(k, priority_order, samples_no_experiment, indexes_no_experiment, "no_experiment")

Intercept 0.46466891562838364
Prediction_local [0.56270333]
Right: 0.72
Intercept 0.5142386962319422
Prediction_local [0.37979484]
Right: 0.93


Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.


Intercept 0.5454733159597056
Prediction_local [0.40637346]
Right: 0.07
Intercept 0.48359576627695067
Prediction_local [0.7458437]
Right: 0.91
Intercept 0.5608347848885814
Prediction_local [0.37889312]
Right: 0.13
Intercept 0.5016704640957175
Prediction_local [0.5886479]
Right: 0.3


Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
